In [1]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import timeit
from numba import njit

In [2]:
# Create a sample pandas Series
np.random.seed(0)
series = pd.Series(np.random.randn(10000))
window = 50

# Benchmark numpy vs pandas

In [6]:
# Rolling mean
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).mean()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.mean(sliding_window_view(series, window), axis=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.028297 seconds, Std Dev: 0.003984 seconds
Numpy Mean Time : 0.018500 seconds, Std Dev: 0.000188 seconds


In [7]:
# Rolling std
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).std()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.std(sliding_window_view(series, window), axis=1, ddof=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.030795 seconds, Std Dev: 0.004558 seconds
Numpy Mean Time : 0.234323 seconds, Std Dev: 0.024244 seconds


In [8]:
# Rolling median
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).median()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.median(sliding_window_view(series, window), axis=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.400690 seconds, Std Dev: 0.010012 seconds
Numpy Mean Time : 0.699383 seconds, Std Dev: 0.010848 seconds


In [9]:
# Rolling multiple stats
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    rolling = series.rolling(window=window)
    stats = [rolling.mean(), rolling.std(), rolling.median(), rolling.max(), rolling.min(), rolling.sum()]
    return stats

# 2. Nunmpy Implementation
def numpy_rolling(series, window):

    rolling = sliding_window_view(series, window)
    stats = [
        np.mean(rolling, axis=1),
        np.std(rolling, axis=1, ddof=1),
        np.median(rolling, axis=1),
        np.max(rolling, axis=1),
        np.min(rolling, axis=1),
        np.sum(rolling, axis=1)
    ]
    return stats

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

Pandas Mean Time: 0.526125 seconds, Std Dev: 0.012349 seconds
Numpy Mean Time : 1.083005 seconds, Std Dev: 0.030257 seconds


# Optimization with numba

Durante el predict recursivo, se tiene que aplicar una función repetitiva sobre un array de numpy.

In [18]:
predictions = [np.random.randn(50) for _ in range(1000)]
predictions_pd = [pd.Series(np.random.randn(50)) for _ in range(1000)]

def numpy_version(predictions):
    for last_window in predictions:
        np.median(last_window)

@njit
def numpy_numba_version(predictions):
    for last_window in predictions:
        np.median(last_window)

def pandas_version(predictions):
    for last_window in predictions:
        pd.Series(last_window).median()

repeats = 5
number = 10
numpy_times = timeit.repeat(lambda: numpy_version(predictions), number=number, repeat=repeats)
numba_times = timeit.repeat(lambda: numpy_numba_version(predictions), number=number, repeat=repeats)
pandas_times = timeit.repeat(lambda: pandas_version(predictions_pd), number=number, repeat=repeats)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
numba_mean = np.mean(numba_times)
numba_std = np.std(numba_times)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")
print(f"Numba Mean Time : {numba_mean:.6f} seconds, Std Dev: {numba_std:.6f} seconds")
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")

Numpy Mean Time : 0.154755 seconds, Std Dev: 0.004867 seconds
Numba Mean Time : 0.110940 seconds, Std Dev: 0.072713 seconds
Pandas Mean Time: 0.607512 seconds, Std Dev: 0.004432 seconds


In [22]:
predictions = [np.random.randn(50) for _ in range(1000)]
predictions_pd = [pd.Series(np.random.randn(50)) for _ in range(1000)]

def numpy_version(predictions):
    for last_window in predictions:
        stats = [
            np.mean(last_window),
            np.std(last_window),
            np.median(last_window),
            np.max(last_window),
            np.min(last_window),
            np.sum(last_window)
        ]

@njit
def numpy_numba_version(predictions):
    for last_window in predictions:
        stats = [
            np.mean(last_window),
            np.std(last_window),
            np.median(last_window),
            np.max(last_window),
            np.min(last_window),
            np.sum(last_window)
        ]

def pandas_version(predictions):
    for last_window in predictions:
        lw = pd.Series(last_window)
        stats = [
            lw.mean(),
            lw.std(),
            lw.median(),
            lw.max(),
            lw.min(),
            lw.sum()
        ]

repeats = 5
number = 10
numpy_times = timeit.repeat(lambda: numpy_version(predictions), number=number, repeat=repeats)
numba_times = timeit.repeat(lambda: numpy_numba_version(predictions), number=number, repeat=repeats)
pandas_times = timeit.repeat(lambda: pandas_version(predictions_pd), number=number, repeat=repeats)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
numba_mean = np.mean(numba_times)
numba_std = np.std(numba_times)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")
print(f"Numba Mean Time : {numba_mean:.6f} seconds, Std Dev: {numba_std:.6f} seconds")
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")

Numpy Mean Time : 0.464398 seconds, Std Dev: 0.016344 seconds
Numba Mean Time : 0.165382 seconds, Std Dev: 0.188210 seconds
Pandas Mean Time: 1.486151 seconds, Std Dev: 0.015396 seconds


In [55]:
series = pd.Series(np.array([np.nan, 2, np.nan, 4, 5, 6, 7, 8, 9, 10]))
series_2 = pd.Series(np.array([1, 2, 3, 4, 5, np.nan, 7, 8, 9, 10]))

In [56]:
pd.DataFrame({'1':series,
              '2': series_2
}).rolling(window=3, min_periods=3, closed='left').mean()

,1,2
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,2.0
4,NaN,3.0
5,NaN,4.0
6,5.0,NaN
7,6.0,NaN
8,7.0,NaN
9,8.0,8.0


In [ ]:
RollingFeatures(
    stats = str, [str],
    window_sizes = int, [int],
    features_names = None, list,
    min_periods=None=window_size, int, [int],
    closed='left', [str],
    # fill_strategy=ffil, [], #warning, ffill no esta disponible durante predict, usa el argument fill_strategy_predict
    # fill_strategy_predict=None, [],
)

transform_batch
transform



# Skforecast

In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [5]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.preprocessing import TimeSeriesDifferentiator

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y as y_categorical
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog as exog_categorical
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation

from skforecast.preprocessing import RollingFeatures

In [23]:
rolling = RollingFeatures(
    stats = ['mean', 'std', 'median', 'max', 'min', 'sum'],
    window_sizes = [5, 6, 7, 6, 5, 5],
    min_periods = [5, 5, 5, 5, 5, 4],
    closed = 'left'
)

In [24]:
print(rolling.stats)
print(rolling.window_sizes)
print(rolling.features_names)
print(rolling.min_periods)
print(rolling.closed)
print(rolling.fill_strategy)
print(rolling.fill_strategy_predict)

['mean', 'std', 'median', 'max', 'min', 'sum']
[5, 6, 7, 6, 5, 5]
None
[5, 5, 5, 5, 5, 4]
['left', 'left', 'left', 'left', 'left', 'left']
[None, None, None, None, None, None]
[None, None, None, None, None, None]


In [25]:
import pprint
pprint.pp(rolling._unique_rolling_windows)

{'5_5_left': {'params': {'window': 5, 'min_periods': 5, 'closed': 'left'},
              'stats_idx': [0, 4],
              'rolling_obj': None},
 '6_5_left': {'params': {'window': 6, 'min_periods': 5, 'closed': 'left'},
              'stats_idx': [1, 3],
              'rolling_obj': None},
 '7_5_left': {'params': {'window': 7, 'min_periods': 5, 'closed': 'left'},
              'stats_idx': [2],
              'rolling_obj': None},
 '5_4_left': {'params': {'window': 5, 'min_periods': 4, 'closed': 'left'},
              'stats_idx': [5],
              'rolling_obj': None}}


In [27]:
series = pd.Series(np.arange(50), name='series')

rolling.transform_batch(series)

{'window': 5, 'min_periods': 5, 'closed': 'left'}
{'window': 6, 'min_periods': 5, 'closed': 'left'}
{'window': 7, 'min_periods': 5, 'closed': 'left'}
{'window': 5, 'min_periods': 4, 'closed': 'left'}


,_series_mean,_series_std,_series_median,_series_max,_series_min,_series_sum
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,6.0
5,2.0,1.581139,2.0,4.0,0.0,10.0
6,3.0,1.870829,2.5,5.0,1.0,15.0
7,4.0,1.870829,3.0,6.0,2.0,20.0
8,5.0,1.870829,4.0,7.0,3.0,25.0
9,6.0,1.870829,5.0,8.0,4.0,30.0
